# 모델 양자화 - 오퍼레이터 합치기
🔍 왜 오퍼레이터를 합치나요?

1. 양자화된 연산은 정밀도가 낮아서 연산 분리 시 오류 누적이 크기 때문  
* int8 연산은 float32보다 표현력이 적음 → 중간 결과 저장이 부정확해짐
* 연산을 나눠두면 중간값이 더 많이 깨지고 정확도 손실이 큼

-> 합치면 중간 결과 없이 한 번에 연산하므로 정확도 손실을 줄일 수 있음

2. 실제 하드웨어에서는 분리된 연산보다 fused 연산이 훨씬 빠름  
* 많은 양자화 하드웨어/엔진은 Conv+ReLU 또는 Conv+BN+ReLU를 하나의 커널로 실행함
* 분리된 연산보다 메모리 접근 횟수가 줄어들고, 캐시 효율도 올라감

In [ ]:
import torch
import torch.nn as nn
import torch.quantization as quant

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 16, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.conv(x))

model = Model()

# 오퍼레이터 합치기 수행
model_fused = torch.quantization.fuse_modules(model, [['conv', 'relu']], inplace=False)